# Installation
What it does: This cell installs the necessary Python libraries that are not pre-installed in the standard Kaggle environment.

langchain-google-genai: The specific connector allowing us to talk to Google's Gemini models.

langgraph: The orchestration library that lets us build the "flowchart" of agents (defining how they talk to each other).

duckduckgo-search: A free, privacy-focused search engine tool that allows the agent to fetch real-time data from the web without needing a complex API setup.

Why it matters for the Competition: This sets the stage for using Gemini (Bonus Points) and Tools (Implementation Criteria).

In [6]:
!pip install -U -q langchain-google-genai langgraph langchain-community duckduckgo-search ddgs

# Imports & Setup
What it does: This cell initializes the environment and connects your code to the Google Gemini API.

Imports: We import ChatGoogleGenerativeAI (the brain), DuckDuckGoSearchRun (the tool), and StateGraph (the body/structure).

Security: It uses UserSecretsClient. Crucially, this prevents you from hard-coding your password (API Key) into the code. When you make your notebook public, your key remains hidden in the Kaggle system.

Model Configuration: We set temperature=0.3. This controls "creativity." A lower number (0.3) makes the agent more factual and focused, which is better for following instructions like creating a shopping list.

Why it matters: Using UserSecretsClient shows professional coding practices (Security). Using gemini-1.5-flash ensures the agent is fast and efficient.

In [7]:
import os
from typing import TypedDict, Annotated, List
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.graph import StateGraph, END
from langchain_core.messages import SystemMessage, HumanMessage

# --- CONFIGURATION ---
# If using Kaggle Secrets, uncomment the lines below:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")

# Initialize the Gemini Model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

# Initialize Search Tool
search_tool = DuckDuckGoSearchRun()

print("Environment Setup Complete.")

Environment Setup Complete.


# State & Agent Definitions
What it does: This is the core logic of your application. It defines the "Memory" and the three specific "Personalities" (Agents).

1. The AgentState (Memory): Think of this as a shared clipboard or a folder that gets passed from person to person. It has four slots:

user_request: What the user said originally.

pantry_items: The list of ingredients extracted.

research_results: The raw text found on the internet.

final_plan: The formatted output.

2. The Agents:

input_analyzer_agent (The Analyst):

Job: detailed extraction.

Logic: It takes a messy sentence ("I've got some old chicken and rice") and turns it into a clean list ['chicken', 'rice']. It saves this to pantry_items.

research_agent (The Researcher):

Job: External Tool Use.

Logic: It looks at the pantry_items and formulates a search query. It then actually goes to the internet using search_tool.invoke. It saves the findings to research_results.

chef_agent (The Chef):

Job: Synthesis and Formatting.

Logic: It reads the pantry_items and the research_results. It uses a strict prompt ("You are an expert Meal Planner") to generate the final response.

Why it matters: This cell satisfies the Multi-Agent System requirement (3 distinct agents) and the Session/State Management requirement (passing the AgentState dictionary).

In [8]:
# --- 1. STATE MANAGEMENT (Memory) ---
# This dictionary acts as the "Session Memory" passed between agents
class AgentState(TypedDict):
    user_request: str
    pantry_items: List[str]
    research_results: str
    final_plan: str

# --- 2. AGENT DEFINITIONS ---

def input_analyzer_agent(state: AgentState):
    """
    Agent 1: Parses the user input to identify pantry items.
    """
    print(f"🕵️  Analyst Agent: Extracting pantry items from '{state['user_request']}'...")
    
    prompt = f"""
    Extract the list of food ingredients/pantry items from this user request: "{state['user_request']}".
    Return ONLY a comma-separated list of ingredients. Do not add conversational text.
    """
    response = llm.invoke(prompt)
    items = [item.strip() for item in response.content.split(',')]
    
    return {"pantry_items": items}

def research_agent(state: AgentState):
    """
    Agent 2: Uses Tools to find recipes based on the pantry items.
    """
    items = ", ".join(state['pantry_items'])
    print(f"🌐 Search Agent: Searching for recipes using {items}...")
    
    query = f"Simple 3-day meal plan recipes using {items} and common staples."
    
    # --- TOOL USE (External Data) ---
    try:
        search_result = search_tool.invoke(query)
    except Exception as e:
        search_result = f"Search failed: {e}. I will generate based on general knowledge."
        
    return {"research_results": search_result}

def chef_agent(state: AgentState):
    """
    Agent 3: Synthesizes research and creates the final plan.
    """
    print("👨‍🍳 Chef Agent: Cooking up the plan...")
    
    prompt = f"""
    You are an expert Meal Planner. 
    
    User's Pantry: {state['pantry_items']}
    Research Context: {state['research_results']}
    
    Task: Create a 3-Day Meal Plan (Lunch & Dinner) that uses the pantry items.
    Format:
    1. A Day-by-Day Menu.
    2. A Shopping List (ONLY for items NOT in the User's Pantry).
    
    Keep it concise and practical.
    """
    response = llm.invoke(prompt)
    return {"final_plan": response.content}

print("Agents Defined.")

Agents Defined.


# The Workflow Graph
What it does: This cell uses LangGraph to wire the agents together. It creates a Directed Acyclic Graph (DAG).

Nodes: We assign names to our functions (e.g., "analyst" runs the input_analyzer_agent function).

Edges: We define the path.

analyst -> researcher

researcher -> chef

chef -> END (Finish execution).

Compile: This builds the executable application (app).

Why it matters: This demonstrates "Architecture" and "Control Flow." Instead of a giant if/else block, you have a modular graph. If you wanted to add a "Nutritionist Agent" later, you would just add a node here.

In [9]:
# --- 3. BUILD THE GRAPH ---
workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("analyst", input_analyzer_agent)
workflow.add_node("researcher", research_agent)
workflow.add_node("chef", chef_agent)

# Add Edges (Linear Logic: Analyst -> Researcher -> Chef -> End)
workflow.set_entry_point("analyst")
workflow.add_edge("analyst", "researcher")
workflow.add_edge("researcher", "chef")
workflow.add_edge("chef", END)

# Compile
app = workflow.compile()

print("Agent Workflow Compiled.")

Agent Workflow Compiled.


# Execution
What it does: This is the user interface.

Input: You define user_input (e.g., "I have eggs and cheese").

Invoke: app.invoke(inputs) starts the graph. The input enters the "Analyst" node, and the AgentState flows through the graph until it hits the "Chef."

Display: It prints the result in Markdown format so it looks nice (bold headers, bullet points).

Why it matters: This is the "Proof of Work." It shows the judges that your code actually runs and produces a useful result based on the concepts you implemented.

In [10]:
from IPython.display import Markdown

# --- INPUT ---
user_input = "I have some leftover chicken, a bag of rice, and spinach. What can I eat?"

# --- EXECUTION ---
inputs = {"user_request": user_input}
result = app.invoke(inputs)

# --- OUTPUT ---
display(Markdown(f"# 🍽️ Pantry-to-Plate Results\n\n{result['final_plan']}"))

🕵️  Analyst Agent: Extracting pantry items from 'I have some leftover chicken, a bag of rice, and spinach. What can I eat?'...
🌐 Search Agent: Searching for recipes using chicken, rice, spinach...
👨‍🍳 Chef Agent: Cooking up the plan...


# 🍽️ Pantry-to-Plate Results

Here is a 3-Day Meal Plan using your pantry items:

### 1. Day-by-Day Menu

**Day 1: Savory One-Pan Chicken & Rice with Spinach**
*   **Lunch:** One-Pan Chicken and Rice with Spinach (cooked with onions for a comforting, savory base)
*   **Dinner:** One-Pan Chicken and Rice with Spinach (leftovers or fresh batch, same savory profile)

**Day 2: Zesty Lemony Chicken & Rice with Spinach**
*   **Lunch:** One-Pan Lemony Chicken and Rice with Spinach (brightened with lemon and lemon pepper seasoning)
*   **Dinner:** One-Pan Lemony Chicken and Rice with Spinach (leftovers or fresh batch, same zesty profile)

**Day 3: Simple & Versatile Chicken & Rice with Spinach**
*   **Lunch:** Simple Chicken and Rice with Spinach (prepared with basic seasonings, allowing the natural flavors to shine. Can use any leftover onions or lemon for a subtle hint.)
*   **Dinner:** Simple Chicken and Rice with Spinach (same as lunch)

### 2. Shopping List (ONLY for items NOT in the User's Pantry)

*   Onions (or onion powder)
*   Lemons
*   Lemon Pepper Seasoning
*   Kosher Salt